## Follow the instructions of this notebook to generate the csv files used in analysis for kiddraw
### If you ran download_data.py, running this code is not necessary

### Import packages and set up paths

In [ ]:
import os, sys
import pymongo as pm
import pandas as pd
import socket
import json
import numpy as np
import utils
import base64
import importlib
import time
from collections import Counter
from PIL import Image
from io import BytesIO
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

In [ ]:
# directory & file hierarchy
proj_dir     = os.path.abspath('..')
analysis_dir = os.getcwd()
results_dir  = os.path.join(proj_dir, 'results')
plot_dir     = os.path.join(results_dir, 'plots')
csv_dir      = os.path.join(results_dir, 'csv')
exp_dir      = os.path.abspath(os.path.join(proj_dir, 'experiments'))
sketch_dir   = os.path.abspath(os.path.join(proj_dir, 'sketches'))
gallery_dir  = os.path.abspath(os.path.join(proj_dir, 'gallery'))

if socket.gethostname() == 'nightingale':
    feature_dir = os.path.abspath('/mnt/pentagon/photodraw/features/')
else:
    feature_dir = os.path.abspath(os.path.join(proj_dir, 'features'))

meta_path      = os.path.abspath(os.path.join(feature_dir, 'metadata_pixels.csv'))
image_path     = os.path.abspath(os.path.join(feature_dir, 'flattened_sketches_pixels.npy'))
meta_path_fc6  = os.path.abspath(os.path.join(feature_dir, 'METADATA_sketch.csv'))
image_path_fc6 = os.path.abspath(os.path.join(feature_dir, 'FEATURES_FC6_sketch_no-channel-norm.npy'))

# add helpers to python path
if os.path.join(proj_dir, 'utils') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'utils'))

def make_dir_if_not_exists(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

## create directories that don't already exist        
result = [make_dir_if_not_exists(x) for x in [results_dir, plot_dir, csv_dir, sketch_dir, gallery_dir, feature_dir]]

### establish connection to mongo

`ssh -fNL 27020:127.0.0.1:27017 jyang@cogtoolslab.org`  
`ssh -fNL 27017:127.0.0.1:27017 jyang@cogtoolslab.org`

In [ ]:
# Setting to true produces S, T, and K directly from mongo
yes = True
if yes:
    # set vars (auth.txt file contains the password for the sketchloop user)
    pswd = auth.values[0][0]
    auth = pd.read_csv(os.path.join(analysis_dir, 'auth.txt'), header=None)
    decoderpswd = int(pswd[-1])
    user = 'sketchloop'
    host = 'cogtoolslab.org'

    # have to fix this to be able to analyze from local
    import socket
    if socket.gethostname().split('_')[0] == 'Justin':
        conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1:27020')
    else:
        conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1:27017')
    db = conn['photodraw']
    coll = db['kiddraw']

    # set iteration name
    iterationName = 'run0'

    # how many records do we have in mongo?
    print(f"We have {coll.estimated_document_count()} records in mongo.")
    print(f"{len(list(coll.find({'iterationName':iterationName, 'eventType':'sketch'})))} of these are sketches.")
    print(f"{len(list(coll.find({'iterationName':iterationName, 'eventType':'stroke'})))} of these are strokes.")

### Initialize, transform dataframes
#### Call data from mongo server and create dataframes with the relevant metadata

In [ ]:
if yes:
    # Sketches
    k = coll.find({'iterationName':iterationName, 'eventType':'sketch'})
    K = pd.DataFrame(k)

    # Strokes
    t = coll.find({'iterationName':iterationName, 'eventType':'stroke'})
    T = pd.DataFrame(t)

    # Surveys
    s = coll.find({'iterationName':iterationName, 'eventType':'survey'})
    S = pd.DataFrame(s).sort_values('aID').reset_index(drop=True)

    responses = []
    coolbeans = ''
    j = 0
    for i in range(len(S.responses)):
        if (i % 2 == 0):
            coolbeans = S.responses[i][:-1] + ', '
            S = S.replace(S['question_order'][i+1], S['question_order'][i])
        else:
            coolbeans = coolbeans + S.responses[i][1:]
            responses.append(json.loads(coolbeans))        
            S = S.drop(S.index[j])
            j+=1
    S = S.reset_index(drop=True)
    responses = pd.DataFrame(responses)
    S = pd.concat([S, responses], axis=1)

In [ ]:
# for encoding and decoding workerIds
def scrambler(messages):
    encoded = []
    for message in messages:
        msg = 'mum' + message + 'sus'
        msg = ''.join(chr(ord(letter)+2) for letter in msg)
        encoded.append(msg)
    return encoded

def descrambler(messages):
    decoded = []
    for message in messages:
        msg = ''.join(chr(ord(letter)-2) for letter in message)
        msg = msg[3::]
        msg = msg[:-3]
        decoded.append(msg)
    return decoded

def encoder(messages):
    message = messages
    for i in range(4):
        message = scrambler(message)
    return message
  
def decoder(messages, password):
    message = messages
    for i in range(password):
        message = descrambler(message)
    return message

if yes:
    # encode workerIDs
    T['workerId'] = encoder(T['workerId'])
    S['workerId'] = encoder(S['workerId'])

# use decoder function to decode workerIDs

In [ ]:
if yes:
    # get list of valid game IDs (i.e, subject number)
    game_dict = Counter(K['gameID'])

    # get gameids that contributed exactly 12 sketches
    complete_gameids = [k for (k, v) in game_dict.items() if v == 12]

    # subset stroke/sketch dataframes by being complete AND also exclude practice
    subset = True
    if (subset and T['gameID'].nunique() != len(complete_gameids)):
        T = T[(T['gameID'].isin(complete_gameids))].reset_index(drop=True)
        K = K[(K['gameID'].isin(complete_gameids))].reset_index(drop=True)

    print('We have {} unique stroke records in all {} of our complete games.'.format(T.shape[0], len(complete_gameids)))
    print('We have {} unique sketch records in all {} of our complete games.'.format(K.shape[0], len(complete_gameids)))

In [ ]:
if yes:
    # add gameIDs to survey data
    keys_gameID = list(T.sort_values('aID')['gameID'].unique())
    values_aID = list(T.sort_values('aID')['aID'].unique())
    values_aID = values_aID[:36] + ['3MX2NQ3YCDUJZ0KA8WJ4KDVBNTOX5Q'] + values_aID[36:]
    dictionary = dict(zip(keys_gameID, values_aID))

    S['gameID'] = list(dictionary.keys())
    K['aID'] = K['gameID'].map(dictionary)

    # add photoids to sketch data
    photoids = []
    for index, row in K.iterrows():
        if row['condition'] == 'photo':
            photoids.append(int(row['imageURL'][-5]))
        else:
            photoids.append('text')
    K = K.assign(photoid=np.asarray(photoids))

In [ ]:
if yes:
    # adds activeSketchTime to K, and sorts the K, T values lexicographically by gameID and then trialNum
    K = K.sort_values(by=['gameID', 'trialNum']).reset_index(drop=True)
    T = T.sort_values(by=['gameID', 'trialNum']).reset_index(drop=True)
    nmnm = []
    for gID in K.gameID.unique():
        temp = T[T['gameID'] == gID]
        for i in temp.trialNum.unique():
            temp2 = temp[temp['trialNum'] == i]
            nmnm.append(temp2['endStrokeTime'].iloc[-1]-temp2['startStrokeTime'].iloc[0])
    K = K.assign(activeSketchTime=nmnm)

    # assigns totalInk to K
    totalInk = pd.DataFrame(T.groupby(['gameID', 'trialNum'])['arcLength'].aggregate(np.sum)).reset_index()['arcLength']
    K = K.assign(totalInk=totalInk)

    # adds flags to K stating whether or not to exclude the sketch from data analysis
    K = utils.preprocess_sketches(K)

    # add labels for category-photoid and category-condition pairs
    K = K.assign(cat_id=K.category + '_' + K.photoid)
    K = K.assign(cat_cond=K.category + '_' + K.condition)

In [ ]:
# convert rgba data to rgb is in rgba, and get everything in the right dimensions
def rgba2rgb(rgba, background=(255, 255, 255)):
    row, col, ch = rgba.shape

    if ch == 3:
        return rgba

    assert ch == 4, 'RGBA image has 4 channels.'

    rgb = np.zeros((row, col, 3), dtype='float32')
    r, g, b, a = rgba[:, :, 0], rgba[:, :, 1], rgba[:, :, 2], rgba[:, :, 3]

    a = np.asarray(a, dtype='float32') / 255.0

    R, G, B = background

    rgb[:, :, 0] = r * a + (1.0 - a) * R
    rgb[:, :, 1] = g * a + (1.0 - a) * G
    rgb[:, :, 2] = b * a + (1.0 - a) * B

    return np.asarray(rgb, dtype='double')


if yes:
    # make an image array of (732x244x244x3) from raw png data
    lis = []
    ist = []
    for i in range(len(K['pngData'])):
        lis.append(np.array(Image.open(BytesIO(base64.b64decode(K['pngData'][i]))).resize((224, 224))))
        ist.append(np.array(rgba2rgb(lis[i])))
    lis = np.array(lis)
    ist = np.array(ist)

    # make a 732 x (224*224*3) feature vector of pixels
    F = np.array([ist[i].flatten() for i in range(len(ist))])

    # load in fc6 feature vectors
    F_fc6 = np.load(image_path_fc6)

    # normalize both feature vectors mean-wise
    F_norm = F - F.mean(axis=0)
    F_fc6_norm = F_fc6 - F_fc6.mean(axis=0)

In [ ]:
if yes:
    classes_map = {'airplane':0, 
                   'bike':1, 
                   'bird':2, 
                   'car':3, 
                   'cat':4, 
                   'chair':5, 
                   'cup':6, 
                   'hat':7, 
                   'house':8, 
                   'rabbit':9, 
                   'tree':10, 
                   'watch':11}
    kFold = KFold(n_splits = 5)
    time0 = time.time()
    logit = LogisticRegression(max_iter=1000)
    sketch_probs_lists = []
    i = 1
    for train_index, test_index in kFold.split(F_fc6_norm):
        model = logit.fit(F_fc6_norm[train_index], K['category'][train_index])
        probarrs = model.predict_proba(F_fc6_norm[test_index])
        actual = K['category'][test_index].values
        probs = [prob[classes_map[act]] for prob, act in zip(probarrs, actual)]
        probarr = np.array(list(zip(test_index, actual, probs)))
        sketch_probs_lists.append(probarr)
        print(f'On loop number {i}. {time.time() - time0} seconds since starting. \r', end="")
        i += 1
    sketch_probs_arrays = np.vstack(sketch_probs_lists)
    pred_probs = pd.DataFrame(sketch_probs_arrays, columns = ['sketch_index', 'cat_codes_true', 'prob_truth_fc6'])

    K = K.assign(prob_true_predict_fc6 = pred_probs.prob_truth_fc6.astype(float))

    # pixel-level features
    time1 = time.time()
    logit = LogisticRegression(max_iter=1000)
    sketch_probs_lists = []
    i = 1
    for train_index, test_index in kFold.split(F_norm):
        model = logit.fit(F_norm[train_index], K['category'][train_index])
        probarrs = model.predict_proba(F_norm[test_index])
        actual = K['category'][test_index].values
        probs = [prob[classes_map[act]] for prob, act in zip(probarrs, actual)]
        probarr = np.array(list(zip(test_index, actual, probs)))
        sketch_probs_lists.append(probarr)
        print(f'On loop number {i}. {time.time() - time0} seconds since starting the script. {time.time() - time1} seconds since starting the pixel-level CV. \r', end="")
        i += 1
    sketch_probs_arrays2 = np.vstack(sketch_probs_lists)
    pred_probs2 = pd.DataFrame(sketch_probs_arrays2, columns = ['sketch_index', 'cat_codes_true', 'prob_truth'])
    K = K.assign(prob_true_predict_pixel = pred_probs2.prob_truth.astype(float))
    K['prob_true_predict_logodds'] = K.prob_true_predict_pixel.transform(lambda p: np.log10(p/(1-p))).values
    K['prob_true_predict_fc6_logodds'] = K.prob_true_predict_fc6.transform(lambda p: np.log10(p/(1-p))).values

    # does every sketch have a valid probability?
    assert all(K.prob_true_predict_fc6 >= 0) & all(K.prob_true_predict_fc6 <= 1)
    assert all(K.prob_true_predict_pixel >= 0) & all(K.prob_true_predict_pixel <= 1)

In [ ]:
if yes:
    # save data out to csv
    T.to_csv(os.path.join(csv_dir, 'photodraw_stroke_data.csv'), index=False)
    K.to_csv(os.path.join(csv_dir, 'photodraw_sketch_data.csv'), index=False)
    S.to_csv(os.path.join(csv_dir, 'photodraw_survey_data.csv'), index=False)
    np.save(image_path, F)
else:
    # read in csv
    T = pd.read_csv(os.path.join(csv_dir, 'photodraw_stroke_data.csv'))
    K = pd.read_csv(os.path.join(csv_dir, 'photodraw_sketch_data.csv'))
    S = pd.read_csv(os.path.join(csv_dir, 'photodraw_survey_data.csv'))
    F = np.load(image_path)
    F_fc6 = np.load(image_path_fc6)

### render out sketches

In [ ]:
importlib.reload(utils)
utils.render_images(K,
                    data='pngData',
                    metadata=['gameID', 'trialNum', 'condition', 'category', 'photoid'],
                    out_dir=sketch_dir)

### Make sketch gallery

In [ ]:
sketch_paths = sorted([sketch_path for sketch_path in os.listdir(sketch_dir)])
gameids = list(np.unique([i.split('_')[0] for i in sketch_paths]))

if (len(os.listdir(gallery_dir)) == 0):
    # generate gallery for each participant
    for gind, game in enumerate(gameids):
        print('Generating sketch gallery for participant: {} | {} of {}'.format(game, gind+1, len(gameids)))
        # get list of all sketch paths JUST from current game
        game_sketch_paths = [path for path in sketch_paths if path.split('_')[0] == game]
        fig = plt.figure(figsize=(8, 12))
        for i, f in enumerate(game_sketch_paths):
            # open image
            im = Image.open(os.path.join(sketch_dir, f))
            # get metadata
            gameid = f.split('_')[0]
            trialNum = f.split('_')[1]
            condition = f.split('_')[2]
            category = f.split('_')[3].split('.')[0]
            # make gallery
            p = plt.subplot(3, 4, i+1)
            plt.imshow(im)
            sns.set_style('white')
            k = p.get_xaxis().set_ticklabels([])
            k = p.get_yaxis().set_ticklabels([])
            k = p.get_xaxis().set_ticks([])
            k = p.get_yaxis().set_ticks([])
            p.axis('off')
            plt.title('{} {} {}'.format(trialNum, condition, category))
        plt.suptitle(gameid)
        fname = '{}.png'.format(gameid)
        plt.savefig(os.path.join(gallery_dir, fname))
        plt.close(fig)
    print('Done!')

### Make flattened pixel data and metadata

In [ ]:
# make an image array of (732x244x244x3) from raw png data
lis = []
ist = []
for i in range(len(K['pngData'])):
    lis.append(np.array(Image.open(BytesIO(base64.b64decode(K['pngData'][i]))).resize((224, 224))))
    ist.append(np.array(rgba2rgb(lis[i])))
lis = np.array(lis)
ist = np.array(ist)

# make a 732 x (224*224*3) feature vector of pixels
F = np.array([ist[i].flatten() for i in range(len(ist))])

# make metadata containing _id, gameID, condition, category, numStrokes, trialNum, and photo-id
photoid = [i[-5] for i in K['imageURL']]
M = np.array(K[['_id', 'gameID', 'trialNum', 'condition', 'category']])
M = np.hstack((M, np.array(photoid).astype(int).reshape(732, 1)))
M = pd.DataFrame(M, columns=['_id', 'gameID', 'trialNum', 'condition', 'category', 'photoid'])
for i in range(len(M)):
    if (M['condition'][i] == 'text'):
        M['photoid'][i] = 'text'
M = M.assign(cat_id=M.category.astype(str) + M.photoid.astype(str))
M = M.assign(cat_cond=M.category + M.condition)
M = M.assign(cond_codes=M.condition.astype('category').cat.codes)
M = M.assign(cat_codes=M.category.astype('category').cat.codes)
M = M.assign(photoid_codes=M.photoid.astype('category').cat.codes)
M = M.assign(cat_cond_codes=M.cat_cond.astype('category').cat.codes)
M = M.assign(cat_id_codes=M.cat_id.astype('category').cat.codes)

M = M.join(K[['isOutlier', 'isInvalid']])

# save out the low-level data
np.save(image_path, F), M.to_csv(meta_path, index=False)

# save out unflattened sketch data if you want to run extended_sketches
unflattened_ones = os.path.abspath(os.path.join(feature_dir, 'unflattened_sketches_pixel.npy'))
np.save(unflattened_ones, ist)

M.head()

In [ ]:
# change to true if the metadata is still in the old format
M_fc6 = pd.read_csv(meta_path_fc6)
F_fc6 = np.load(image_path_fc6)

if 'cat_cond_codes' not in M_fc6.columns.values:
    temp = []
    for index, row in M_fc6.iterrows():
        temp.append(np.array(row['sketch_id'].split('\\')[1].split('_')))
    M_fc6 = pd.DataFrame(np.vstack(tuple(temp)), columns=['gameID', 'trialNum', 'condition', 'category', 'photoid'])
    M_fc6['trialNum'] = M_fc6.trialNum.astype(int)
    M_fc6 = M_fc6.assign(cat_id=M_fc6.category + M_fc6.photoid)
    M_fc6 = M_fc6.assign(cat_id_codes=M_fc6.cat_id.astype('category').cat.codes)
    M_fc6 = M_fc6.assign(cat_codes=M_fc6.category.astype('category').cat.codes)
    M_fc6 = M_fc6.assign(cat_cond_codes=(M_fc6.category + M_fc6.condition).astype('category').cat.codes)

    F_fc6 = F_fc6[M_fc6.sort_values(['gameID', 'trialNum']).index]
    M_fc6 = M_fc6.sort_values(['gameID', 'trialNum']).reset_index(drop=True)

    M_fc6 = M_fc6.join(K[['isOutlier', 'isInvalid']])

    M_fc6.to_csv(meta_path_fc6, index=False)
    np.save(image_path_fc6, F_fc6)
else:
    print('Up to date!')
    pass
M_fc6.head()

### go to command line and nagivate to analysis directory, run the following for fc6 features

`python extract_features.py --data=/photodraw/sketches/ --layer_ind=5 --data_type=sketch --spatial_avg=True --channel_norm=False --out_dir=/photodraw/features/` 

Turns out you don't need to do this since the features are already extracted and saved directly on github

In [ ]:
run = True
if run:
    M_fc6 = pd.read_csv(os.path.join(feature_dir, "photodraw12/METADATA_photodraw_sketch.csv"))
    F_fc6 = np.load(os.path.join(feature_dir, "photodraw12/FEATURES_FC6_photodraw_sketch.npy"))
    K = pd.read_csv(os.path.join(csv_dir, 'photodraw_sketch_data.csv'))

    if 'cat_cond_codes' not in M_fc6.columns.values:
        temp = []
        for index, row in M_fc6.iterrows():
            temp.append(np.array(row['image_id'].split('\\')[1].split('_')))
        M_fc6 = pd.DataFrame(np.vstack(tuple(temp)), columns=['gameID', 'trialNum', 'condition', 'category', 'photoid'])
        M_fc6['trialNum'] = M_fc6.trialNum.astype(int)
        M_fc6 = M_fc6.assign(cat_id=M_fc6.category + M_fc6.photoid)
        M_fc6 = M_fc6.assign(cat_id_codes=M_fc6.cat_id.astype('category').cat.codes)
        M_fc6 = M_fc6.assign(cat_codes=M_fc6.category.astype('category').cat.codes)
        M_fc6 = M_fc6.assign(cat_cond_codes=(M_fc6.category + M_fc6.condition).astype('category').cat.codes)

        F_fc6 = F_fc6[M_fc6.sort_values(['gameID', 'trialNum']).index]
        M_fc6 = M_fc6.sort_values(['gameID', 'trialNum']).reset_index(drop=True)

        M_fc6 = M_fc6.join(K[['isOutlier', 'isInvalid']])

        M_fc6.to_csv(os.path.join(feature_dir, "photodraw12/METADATA_photodraw_sketch.csv"), index=False)
        np.save(os.path.join(feature_dir, "photodraw12/FEATURES_FC6_photodraw_sketch.npy"), F_fc6)

    F = np.load(image_path)
    F_norm = F - F.mean(axis=0)
    F_fc6_norm = F_fc6 - F_fc6.mean(axis=0)

In [ ]:
# extra code that might become relevant again
classes_map = {'airplane':0, 
               'bike':1, 
               'bird':2, 
               'car':3, 
               'cat':4, 
               'chair':5, 
               'cup':6, 
               'hat':7, 
               'house':8, 
               'rabbit':9, 
               'tree':10, 
               'watch':11}
kFold = KFold(n_splits = 5)
time0 = time.time()
logit = LogisticRegression(max_iter=1000)
sketch_probs_lists = []
sketch_preds_lists = []
i = 1
for train_index, test_index in kFold.split(F_fc6_norm):
    model = logit.fit(F_fc6_norm[train_index], K['category'][train_index])
    probarrs = model.predict_proba(F_fc6_norm[test_index])
    preds = model.predict(F_fc6_norm[test_index])
    actual = K['category'][test_index].values
    probs = [prob[classes_map[act]] for prob, act in zip(probarrs, actual)]
    probarr = np.array(list(zip(test_index, actual, probs)))
    predarr = np.array(list(zip(test_index, actual, preds)))
    sketch_probs_lists.append(probarr)
    sketch_preds_lists.append(predarr)
    print(f'On loop number {i}. {time.time() - time0} seconds since starting. \r', end="")
    i += 1
sketch_probs_arrays = np.vstack(sketch_probs_lists)
sketch_preds_arrays = np.vstack(sketch_preds_lists)
pred_probs = pd.DataFrame(sketch_probs_arrays, columns = ['sketch_index', 'cat_codes_true', 'prob_truth_fc6'])
preds = pd.DataFrame(sketch_preds_arrays, columns = ['sketch_index', 'cat_codes_true', 'truth_fc6'])

K = K.assign(prob_true_predict_fc6 = pred_probs.prob_truth_fc6.astype(float))
K = K.assign(true_predict_fc6 = preds.truth_fc6 == preds.cat_codes_true)

# pixel-level features
time1 = time.time()
logit = LogisticRegression(max_iter=1000)
sketch_probs_lists = []
sketch_preds_lists = []
i = 1
for train_index, test_index in kFold.split(F_norm):
    model = logit.fit(F_norm[train_index], K['category'][train_index])
    probarrs = model.predict_proba(F_norm[test_index])
    actual = K['category'][test_index].values
    probs = [prob[classes_map[act]] for prob, act in zip(probarrs, actual)]
    preds = model.predict(F_norm[test_index])
    probarr = np.array(list(zip(test_index, actual, probs)))
    predarr = np.array(list(zip(test_index, actual, preds)))
    sketch_probs_lists.append(probarr)
    sketch_preds_lists.append(predarr)
    print(f'On loop number {i}. {time.time() - time0} seconds since starting the script. {time.time() - time1} seconds since starting the pixel-level CV. \r', end="")
    i += 1
sketch_probs_arrays2 = np.vstack(sketch_probs_lists)
sketch_preds_arrays2 = np.vstack(sketch_preds_lists)
pred_probs2 = pd.DataFrame(sketch_probs_arrays2, columns = ['sketch_index', 'cat_codes_true', 'prob_truth'])
preds2 = pd.DataFrame(sketch_preds_arrays2, columns = ['sketch_index', 'cat_codes_true', 'truth_pixels'])
K = K.assign(prob_true_predict_pixel = pred_probs2.prob_truth.astype(float))
K = K.assign(true_predict_pixel = preds2.truth_pixels == preds2.cat_codes_true)
K['prob_true_predict_logodds'] = K.prob_true_predict_pixel.transform(lambda p: np.log10(p/(1-p))).values
K['prob_true_predict_fc6_logodds'] = K.prob_true_predict_fc6.transform(lambda p: np.log10(p/(1-p))).values

# does every sketch have a valid probability?
assert all(K.prob_true_predict_fc6 >= 0) & all(K.prob_true_predict_fc6 <= 1)
assert all(K.prob_true_predict_pixel >= 0) & all(K.prob_true_predict_pixel <= 1)